### 1.1 Implémentation du Chiffrement par Décalage (+3) - Chiffre de César
Nous allons maintenant implémenter un chiffrement de type chiffre de César avec un décalage de +3, en utilisant des blocs de 4 caractères.

#### Fonction `block_cipher_encrypt`
Cette fonction recevra le texte à chiffrer et appliquera le décalage de +3 pour chaque caractère dans des blocs de 4.

#### Fonction `block_cipher_decrypt`
La fonction de déchiffrement prendra le texte chiffré et utilisera un décalage inverse (-3) pour restaurer le texte original.

#### Exemple :
Si le message est "HELLO", chaque lettre sera décalée :
- H -> K
- E -> H
- L -> O
- L -> O
- O -> R

Le texte chiffré devient "KHOOR".


In [93]:

# Fonction pour chiffrer par décalage de +3 (Chiffre de César)
def block_cipher_encrypt(plaintext, shift=3, block_size=4):
    
    padding = block_size - len(plaintext) % block_size
    plaintext += ' ' * padding  
    
    ciphertext = ""
    
    
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        encrypted_block = ""
        for char in block:
            
            if char.isalpha(): 
                shifted_char = chr(((ord(char.upper()) - 65 + shift) % 26) + 65)
                encrypted_block += shifted_char
            else:
                encrypted_block += char 
        ciphertext += encrypted_block
    return ciphertext

# Fonction pour déchiffrer le texte avec un décalage inverse
def block_cipher_decrypt(ciphertext, shift=3, block_size=4):
    plaintext = ""
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        decrypted_block = ""
        for char in block:
            if char.isalpha():
                shifted_char = chr(((ord(char.upper()) - 65 - shift) % 26) + 65)
                decrypted_block += shifted_char
            else:
                decrypted_block += char
        plaintext += decrypted_block
    return plaintext


message = "HELLO"
encrypted = block_cipher_encrypt(message)
decrypted = block_cipher_decrypt(encrypted)


print(f"Message : {message}")
print(f"Chiffré : {encrypted}")
print(f"Déchiffré : {decrypted}")


Message : HELLO
Chiffré : KHOOR   
Déchiffré : HELLO   


### 2.1 Chiffrement par XOR avec un flux de clé statique
Dans cette partie de l'exercice, nous allons implémenter un chiffrement symétrique par XOR utilisant une clé statique. Le même flux de clé sera utilisé pour chaque caractère du message.

#### Fonction `stream_cipher_encrypt`
Cette fonction recevra le texte à chiffrer et appliquera un XOR avec la clé sur chaque caractère.

#### Fonction `stream_cipher_decrypt`
La fonction de déchiffrement prendra le texte chiffré et appliquera le même XOR pour récupérer le texte original.

#### Exemple :
Si le message est "HELLO" et que la clé est `[7, 3, 5, 2, 6]`, le chiffrement XOR changera chaque caractère en fonction de la clé et donnera un texte chiffré.


In [94]:
# Fonction de chiffrement XOR

def stream_cipher_encrypt(plaintext, key_stream):
    ciphertext = []
    for i, char in enumerate(plaintext):
        encrypted_char = chr(ord(char) ^ key_stream[i % len(key_stream)])
        ciphertext.append(encrypted_char)
    return ''.join(ciphertext)

# Fonction de déchiffrement XOR
def stream_cipher_decrypt(ciphertext, key_stream):
    plaintext = []
    for i,char in enumerate(ciphertext):
        decrypted_char = chr(ord(char) ^ key_stream[i % len(key_stream)])
        plaintext.append(decrypted_char)
    return ''.join(plaintext)

message = "HELLO"
key_stream = [7, 3, 5, 2, 6]
encrypted = stream_cipher_encrypt(message, key_stream)
decrypted = stream_cipher_decrypt(encrypted, key_stream)

print(f"Message : {message}")
print(f"Chiffré : {''.join([str(ord(c)) for c in encrypted])} (en bytes)")
print(f"Déchiffré: {decrypted}")

Message : HELLO
Chiffré : 7970737873 (en bytes)
Déchiffré: HELLO


###  Exercice 3.1 – Partie 3 : Implémentation du mini-DES (niveau avancé)

Cette partie vise à implémenter un chiffrement de type DES simplifié (mini-DES) sans utiliser de bibliothèques cryptographiques.

Nous allons appliquer un schéma en 8 rounds de type Feistel, avec :

- Permutation Initiale (IP)
- Découpage en blocs (gauche et droite)
- Génération de sous-clés à partir d’une clé principale
- Rounds de chiffrement avec expansion, XOR, S-box, permutation
- Permutation Finale (FP)

Toutes les opérations sont basées sur des manipulations de bits et des tableaux simples.



In [99]:
# Fonctions utilitaires
def permute(bits, table):
    return [bits[i] for i in table]

def xor(bits1, bits2):
    return [b1 ^ b2 for b1, b2 in zip(bits1, bits2)]

def to_bits(n, length=8):
    return [int(b) for b in format(n, f'0{length}b')]

def from_bits(bits):
    return int(''.join(str(b) for b in bits), 2)

In [100]:
# Tables
IP = [1, 5, 2, 0, 3, 7, 4, 6]
FP = [3, 0, 2, 4, 6, 1, 7, 5]
EXPANSION = [3, 0, 1, 2, 1, 2]

SBOX = {
    (0, 0, 0): [0, 1],
    (0, 0, 1): [1, 0],
    (0, 1, 0): [1, 1],
    (0, 1, 1): [0, 0],
    (1, 0, 0): [0, 0],
    (1, 0, 1): [1, 1],
    (1, 1, 0): [1, 0],
    (1, 1, 1): [0, 1],
}

In [101]:
def generate_subkeys(key_bits, rounds=8):
    subkeys = []
    for i in range(rounds):
        rotated = key_bits[i:] + key_bits[:i]
        subkeys.append(rotated[:6])
    return subkeys

def feistel(right, subkey):
    expanded = permute(right, EXPANSION)
    xored = xor(expanded, subkey)
    left_half = xored[:3]
    right_half = xored[3:]
    return SBOX[tuple(left_half)] + SBOX[tuple(right_half)]

In [102]:
def mini_des_encrypt(block_int, key_int, rounds=8):
    block = to_bits(block_int)
    key_bits = to_bits(key_int)
    block = permute(block, IP)
    L, R = block[:4], block[4:]
    subkeys = generate_subkeys(key_bits, rounds)
    for i in range(rounds):
        F = feistel(R, subkeys[i])
        new_R = xor(L, F)
        L, R = R, new_R
    final_block = permute(R + L, FP)
    return from_bits(final_block)

def mini_des_decrypt(cipher_int, key_int, rounds=8):
    block = to_bits(cipher_int)
    key_bits = to_bits(key_int)
    block = permute(block, IP)
    L, R = block[:4], block[4:]
    subkeys = generate_subkeys(key_bits, rounds)
    for i in reversed(range(rounds)):
        F = feistel(R, subkeys[i])
        new_R = xor(L, F)
        L, R = R, new_R
    final_block = permute(R + L, FP)
    return from_bits(final_block)

In [103]:
plaintext = ord('A')  # ASCII 'A' = 65
key = 0b10101010      # clé de 8 bits

cipher = mini_des_encrypt(plaintext, key)
decrypted = mini_des_decrypt(cipher, key)

print(f"Plaintext : {plaintext} ('{chr(plaintext)}')")
print(f"Chiffré   : {cipher}")
print(f"Déchiffré : {decrypted} ('{chr(decrypted)}')")


Plaintext : 65 ('A')
Chiffré   : 73
Déchiffré : 65 ('A')
